In [5]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(filename)s:%(lineno)d - %(message)s",
)

from app.instructions import global_tools_hub
global_tools_hub.load_tools("tools")

2025-02-19 15:11:56,615 - INFO - tools.py:82 - Loading module report_utils from report_utils.py
2025-02-19 15:11:56,615 - INFO - tools.py:82 - Loading module smart_retrieve from smart_retrieve.py
2025-02-19 15:11:56,615 - INFO - tools.py:104 - Registered tool 'retrieve_knowledge_graph' from smart_retrieve.py
2025-02-19 15:11:56,615 - INFO - tools.py:82 - Loading module llm from llm.py
2025-02-19 15:11:56,616 - INFO - tools.py:104 - Registered tool 'llm_generate' from llm.py
2025-02-19 15:11:56,616 - INFO - tools.py:82 - Loading module retrieve from retrieve.py
2025-02-19 15:11:56,616 - INFO - tools.py:104 - Registered tool 'vector_search' from retrieve.py


In [6]:
from notebooks.plan_mcts_optimizer import MCTSPlanOptimizer

# task_id = pending_tasks[0]['id']
task_id = "2dc87c69-fd03-4c62-93d8-97a3a368e0d8"

optimizer = MCTSPlanOptimizer(task_id=task_id, max_iterations=3, time_limit_seconds=900)

2025-02-19 15:11:59,312 - INFO - plan_mcts_optimizer.py:343 - add child node: 7b180c37bb8146ffa412ce11c39c8658(-1) ---> 8d6532914078487f9ce20be4476f83de(0)
2025-02-19 15:11:59,313 - INFO - plan_mcts_optimizer.py:343 - add child node: 8d6532914078487f9ce20be4476f83de(0) ---> 8cf2942c1af548dab8265fd123e2522c(1)


In [7]:
print("optimizing task", task_id)

def print_node(node):
    print("*" * 100)
    print(node.state.seq_no, node.state.commit_hash, node.visits, node.value)
    for suggestion in node.optimization_suggestions:
        print(suggestion["branch_name"])
        print(suggestion["suggestion"])
    for child in node.children:
        print_node(child)

print_node(optimizer.root)

optimizing task 2dc87c69-fd03-4c62-93d8-97a3a368e0d8
****************************************************************************************************
-1 7b180c37bb8146ffa412ce11c39c8658 0 0.0
****************************************************************************************************
0 8d6532914078487f9ce20be4476f83de 0 0.0
****************************************************************************************************
1 8cf2942c1af548dab8265fd123e2522c 0 0.0


In [8]:
optimizer.optimize()

2025-02-19 15:12:02,949 - INFO - plan_mcts_optimizer.py:418 - selected node: 8cf2942c1af548dab8265fd123e2522c(1)
2025-02-19 15:12:03,369 - INFO - plan_mcts_optimizer.py:463 - Re-execute node: 8cf2942c1af548dab8265fd123e2522c(1)
2025-02-19 15:12:04,774 - INFO - simple_cache.py:67 - No close matches found for goal: what are the specific advantages and challenges of deploying the pd component in microservices mode in tidb
2025-02-19 15:12:09,778 - INFO - _base_client.py:1092 - Retrying request to /chat/completions in 0.399439 seconds
2025-02-19 15:12:15,183 - INFO - _base_client.py:1092 - Retrying request to /chat/completions in 0.946849 seconds
2025-02-19 15:12:18,304 - INFO - _client.py:1038 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-19 15:12:18,310 - INFO - manager.py:164 - Label path: [{'label': 'Complex Task Planning'}, {'label': 'Research & Analysis'}, {'label': 'Technical Design'}] for task What are the specific advantages and challeng

KeyboardInterrupt: 

In [ ]:
answers = optimizer.sort_final_answers()
answers

In [ ]:
from notebooks.tasks import save_best_plan_from_url

save_best_plan_from_url(task_id=task_id, commit_hash=answers[0]['commit_hash'])
# save_best_plan_from_url(task_id="c255c42b-c85d-450f-aa2b-b101f37adef8", commit_hash="53c56f51425643e296ac04bdebe751d1")

In [ ]:
from notebooks.tasks import record_evaluation

record_evaluation(task_id, "APPROVED", f"The plan is optimized by the LLM, and choose the best answer among {len(answers)} answers.")

In [ ]:
from notebooks.tasks import ts
from app.config.database import SessionLocal

tt_id = '5fab14aa-83b1-4e67-951f-0dc3932427c1'
keep_branches = ['main']
with SessionLocal() as session:
    tt = ts.get_task(session, tt_id)
    branches = tt.get_branches()
    for branch in branches:
        if branch['name'] not in keep_branches:
            print(f"deleting branch {branch['name']}")
            tt.delete_branch(branch['name'])
        else:
            print(f"keeping branch {branch['name']}")